In [1]:
import gym
import fh_ac_ai_gym

wumpus_env = gym.make('Wumpus-v0')

In [109]:
class HKB:
    def __init__(self):
        self.rules = []
        self.literals = set()
        
    def tell(self, premise, conclusion):
        self.rules.append(premise + ' => ' + conclusion)
        
    def setH(self, event, pq, posx, posy):
        return 0
        
    def ask(self, q):
        print(self.PLFCEntails(q))
        
    def PLFCEntails(self, q):
        count = {}
        for rule in self.rules:
            premise = self.getPremise(rule)
            self.literals.add(self.getConclusion(rule))
            if premise != '':
                count.update({premise : int(self.count_and_addLiterals(premise))})
        inferred = {}
        for i in self.literals:
            inferred.update({i : False})
        agenda = self.setup_agenda()
        
        while len(agenda)>0:
            p = agenda.pop()
            if p == q:
                return True
            
            if inferred[p] == False:
                inferred[p] = True
                for rule in self.rules:
                    if self.isin(rule, p):
                        count[self.getPremise(rule)] = count[self.getPremise(rule)] - 1
                        if count[self.getPremise(rule)] == 0:
                            agenda.add(self.getConclusion(rule))
            
        return False
            
        
    def count_and_addLiterals(self, premise):
        literals = (premise.split(','))
        for i in literals:
            #save literals in a set
            self.literals.add(i)
        
        
        return len(literals)
    
    def setup_agenda(self):
        agenda = set()
        for rule in self.rules:
            #get rules without premise
            if self.getPremise(rule) == '':
                agenda.add(self.getConclusion(rule))
        #return symbols that are known as true initially
        return agenda
        
    def getPremise(self, rule):
        s = (rule.split(' => '))
        return s[0]
        
    def getConclusion(self, rule):
        s = (rule.split(' => '))
        return s[1]
    
    def isin(self, rule, literal):
        
        premise = self.getPremise(rule)
        literals = (premise.split(','))
        for i in literals:
            if i == literal:
                return True
        return False
        

In [111]:
class KB:
    def __init__(self):
        self.HKB = HKB()
        self.visited = {}
        self.rules = []
        self.HKB.tell('', '-P00')
        self.HKB.tell('', '-W00')
        self.wumpus_env = wumpus_env
        self.setHorn()
    def run(self):
        t = 0
        step = ""
        self.wumpus_env.reset()
        obs, reward,done, info = self.wumpus_env.step(1)
        while(step != "exit"):
            
            self.wumpus_env.render()
            
            self.identify_event(obs)
            
            
            ask = input("ASK: ")
            if ask!="n":
                self.HKB.ask(ask)
            
            step = input("What do you want to do? ")
            if(step=="walk" or step=='w'):                    
                obs, reward,done, info = self.wumpus_env.step(0)
            elif (step=='turn left' or step=='l'):
                obs, reward,done, info = self.wumpus_env.step(1)
            elif (step=='turn right' or step=='r'):
                obs, reward,done, info = self.wumpus_env.step(2)
            elif (step=='grab' or step=='g'):
                obs, reward,done, info = self.wumpus_env.step(3)
            elif (step=='shoot' or step=='s'):
                obs, reward,done, info = self.wumpus_env.step(4)
            elif (step=='climb' or step=='c'):
                obs, reward,done, info = self.wumpus_env.step(5)
            elif (step!='exit' and step!='q' and step!='e' and step.isdigit()):
                if(int(step)<6 and int(step)>0):
                    obs, reward,done, info = self.wumpus_env.step(int(step))
            else: #test
                break
                
                
            print(obs)
            print(info)
            t = t+1
        
            
    def identify_event(self, obs):
        pos = '' + str(obs['x']) + str(obs['y'])
        
        if pos in self.visited.keys():
            return
        
        self.HKB.tell('', '-P' + str(obs['x']) + str(obs['y']) + '')
        self.HKB.tell('', '-W' + str(obs['x']) + str(obs['y']) + '')
        if(obs['breeze'] == True):
            self.HKB.tell('', 'B' + str(obs['x']) + str(obs['y']) + '')
            self.HKB.setH('B', 'P', str(obs['x']), str(obs['y']))
        else:#test
            self.HKB.tell('', '-B' + str(obs['x']) + str(obs['y']) + '')
            self.HKB.setH('B', 'P', str(obs['x']), str(obs['y']))
                          
        if(obs['stench'] == True):
            self.HKB.tell('', 'S' + str(obs['x']) + str(obs['y']) + '')
            self.HKB.setH('S', 'W', str(obs['x']), str(obs['y']))
        else: 
            self.HKB.tell('', '-S' + str(obs['x']) + str(obs['y']) + '')
            self.HKB.setH('S', 'W', str(obs['x']), str(obs['y']))
                          
        self.visited.update({str(pos) : 'visited'})
                
     
    # Benachbarte Felder
    def get_surrounding_fields(self, posx, posy):
        surr = {}
        if posx>=1:
            surr.update({posx-1 : posy})
        if posy>=1:
            surr.update({posx : posy-1})
            
        surr.update({posx+1 : posy})
        surr.update({posx : posy+1})
        
        return surr
    
    def setHorn(self):
        for i in range(4):
            for j in range(4):
                
                surr = self.get_surrounding_fields(i, j)
                self.setHRules(surr, i, j)
                    
                #self.HKB.tell(s[:len(s)-1], 'P' + str(i) + str(j))
                
    def setHRules(self, surr, cx, cy):
        ls = ['S', 'B']
        xs = ['W', 'P']
        for i in range(2):
            for x in surr:
                s = ''
                conclusion = '' + str(xs[i]) + str(x) + str(surr[x])
                s = s + str(ls[i]) + str(cx) + str(cy) + ','
                for y in surr:
                    if y != x:
                        s = s + '-' + str(xs[i]) + str(y) + str(surr[y]) + ','
                self.HKB.tell(s[:len(s)-1], conclusion)
        
KB = KB()
KB.run()

+---+---+---+---+
|   |   |W  |   |
|   |   |   |   |
+---+---+---+---+
|   |   |  G|   |
|   |   |   |   |
+---+---+---+---+
| P |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   | P |
| A^|   |   |   |
+---+---+---+---+
Perception [ St: False, Br: True, G: False, Bu: False, Sc: False ]
Score : -1
ASK: r
False
What do you want to do? r
{'x': 0, 'y': 0, 'gold': False, 'direction': <Direction.EAST: 1>, 'arrow': True, 'stench': False, 'breeze': True, 'glitter': False, 'bump': False, 'scream': False}
{'info', 'no further information'}
+---+---+---+---+
|   |   |W  |   |
|   |   |   |   |
+---+---+---+---+
|   |   |  G|   |
|   |   |   |   |
+---+---+---+---+
| P |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   | P |
| A>|   |   |   |
+---+---+---+---+
Perception [ St: False, Br: True, G: False, Bu: False, Sc: False ]
Score : -2
ASK: n
What do you want to do? w
{'x': 1, 'y': 0, 'gold': False, 'direction': <Direction.EAST: 1>, 'arrow': True, 'stench': False, 'breeze